In [45]:
import pandas as pd

from sqlalchemy import create_engine

import pymysql
pymysql.install_as_MySQLdb()

# Change the following variables to match the season and week you need

In [46]:
season = 2018
week = 16

# Extraction of Data

In [47]:
#Extracting ESPN Projections


position_ids = [0,2,4,6] #qb=0 rb=2 wr=4 te=6
pages = [0,40]

dummy = pd.DataFrame()
for position in position_ids:
    for page in pages:
        url = f"http://games.espn.com/ffl/tools/projections?&slotCategoryId={position}&scoringPeriodId={week}&seasonId={season}&startIndex={page}"
        df = pd.read_html(url,header=1)
        df = df[1]
        if position == 0:
            pos = "QB"
        if position == 2:
            pos = "RB"
        if position == 4:
            pos = "WR"
        if position == 6:
            pos = "TE"
        df['POSITION'] = pos
        df['SEASON'] = season
        df['WEEK'] = week
        if position == 0 and page == 0:
            main = pd.concat([dummy,df])
        else:
            main = pd.concat([main,df])


In [48]:
#Extracting Site 2 
csv_file = f'Resources/Site_2_Projections/Site_2_W{week}_projections.csv'
four44_df = pd.read_csv(csv_file)

In [49]:
#Extracting Fantasy Sharks
csv_file = f'Resources/Fantasy_Sharks_Projections/W{week}_Projections.csv'

#Resources\Fantasy_Sharks_Projections\fantasysharks_W16_Projections.csv
sharks_df = pd.read_csv(csv_file)

# Transformation of Data

In [50]:
#transforming ESPN Data
df_clean = main['PLAYER, TEAM POS'].str.split(',',expand=True)
main['Name'] = df_clean[0]
main = main.drop(columns=['PLAYER, TEAM POS'],axis=1)
main['ESPN'] = main['PTS'] + main['INT']
main.head()

,OPP,STATUS ET,C/A,YDS,TD,INT,RUSH,YDS.1,TD.1,REC,YDS.2,TD.2,PTS,POSITION,SEASON,WEEK,Name,ESPN
0,@Sea,Sun 8:20,24.2/37.5,301.00,2.50,0.80,3.80,15.70,0.20,0.00,0.00,0,22.70,QB,2018,16,Patrick Mahomes,23.50
1,NYG,Sun 1:00,24.6/37.8,271.60,2.40,1.10,2.90,13.00,0.10,0.00,0.00,0,19.90,QB,2018,16,Andrew Luck,21.00
2,KC,Sun 8:20,18.1/27.5,217.60,2.30,0.50,4.30,23.90,0.10,0.00,0.00,0,19.80,QB,2018,16,Russell Wilson,20.30
3,@NYJ,Sun 1:00,23.4/37.7,284.80,1.80,0.30,3.00,16.70,0.10,0.00,0.00,0,19.80,QB,2018,16,Aaron Rodgers,20.10
4,TB,Sun 1:00,21.3/32.4,246.50,1.70,0.60,4.40,23.90,0.30,0.00,0.00,0,19.70,QB,2018,16,Dak Prescott,20.30


In [51]:
#putting ESPN data into CSV
main.to_csv(f'resources/espn_projections/espn-week{week}-projections.csv')

In [52]:
#transforming Site 2

# Applying Fanduel uniform scoring system to Fantasy Sharks raw predictions and creating a new column with the results
four44_df['Site 2'] = four44_df['Pass Yds'] * .04 + four44_df['Rush Yds'] * .1 + four44_df['Rec Yds'] * .1 + \
four44_df['Pass TD'] * 4 + four44_df['Rush TD'] * 6 + four44_df['Rec TD'] * 6 + four44_df['Rec'] * .5 + four44_df['INT'] * -1 \
+ four44_df['Fum'] * -2

#dropping unneeded columns
four44_df = four44_df.drop(['PID','Pos','Team','Opp','aFPA','aFPA Rk','FFPts','Comp','Pass Att','Pass Yds','Pass TD','INT',
'Rush Att','Rush Yds','Rush TD','Rec','Rec Yds','Rec TD','Pa1D','Ru1D','Rec1D','Fum','XP','FG','Grade'], axis=1)

# Rename player column to "Name"
four44_df.rename(columns = {"Player": "Name"}, inplace = True)

In [53]:
#transforming Fantasy Sharks

# Splitting the name into 2 seperate columns
sharks_df[['Last Name','First Name']] = sharks_df['Name'].str.split(', ',expand=True)

# Recombine names into one column in the order of First Last
sharks_df['Name'] = sharks_df['First Name'].str.cat(sharks_df['Last Name'],sep=" ")

# Applying Fanduel uniform scoring system to Fantasy Sharks raw predictions and creating a new column with the results
sharks_df['Fantasy Sharks'] = sharks_df['Yards'] * .04 + sharks_df['Yards.1'] * .1 + sharks_df['Yards.2'] * .1 + \
sharks_df['TD'] * 4 + sharks_df['TD.1'] * 6 + sharks_df['TD.2'] * 6 + sharks_df['Rec'] * .5 + sharks_df['Int'] * -1 

#Removing unneeded columns
sharks_df = sharks_df.drop(['ID', 'Rank', 'First Name', 'Last Name', 'Comp', 'Yards', 'TD', 'TD.1', 
'TD.2', 'Int', 'Att', 'Yards.1', 'Yards.2', 'Rec', 'Fantasy Points'], axis=1)


In [54]:
#combining cleaned DataFrames into one DataFrame

combined_df = sharks_df.merge(four44_df, on=['Name'], sort=False)
combined_df = combined_df.merge(main, on=['Name'], sort=False)

combined_df.round({'Fantasy Sharks': 1, 'Site 2': 1})


pd.options.display.float_format = '{:,.2f}'.format

combined_df = combined_df[['Season', 'Week', 'Name', 'Pos', 'Team', 'Opp', 'Fantasy Sharks', 'Site 2','ESPN']]

combined_df.head()

,Season,Week,Name,Pos,Team,Opp,Fantasy Sharks,Site 2,ESPN
0,2018,16,Patrick Mahomes,QB,KCC,SEA,28.26,22.66,23.50
1,2018,16,Russell Wilson,QB,SEA,KCC,23.13,20.87,20.30
2,2018,16,Matt Ryan,QB,ATL,CAR,21.33,19.17,19.50
3,2018,16,Christian McCaffrey,RB,CAR,ATL,20.24,21.55,22.70
4,2018,16,Ben Roethlisberger,QB,PIT,NOS,21.18,19.57,20.10


In [57]:
combined_df.to_csv(f'Output/Week {week} projections.csv')